In [3]:
import torch
import pandas as pd
import numpy as np
from transformers import BertForSequenceClassification, AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

df_train = pd.read_csv('dataset/train_essays.csv')


def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

dataset = Dataset.from_pandas(df_train)
dataset = dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/1378 [00:00<?, ? examples/s]